In [ ]:
# Import the general functions for metabolic modelling
%run 'general_functions.ipynb'

In [ ]:
# model_wt = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_WT.xml")
# model_oxr = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_OxR.xml")
# model_dmso = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_WT+DMSO.xml")
# model_rur = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_RuR+DMSO.xml")
# models = [model_wt,model_oxr,model_dmso,model_rur]

model_wt = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_WT.xml")
model_oxr = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_OxR.xml")
model_dmso = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_WT+DMSO.xml")
model_rur = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_RuR+DMSO.xml")
models = [model_wt,model_oxr,model_dmso,model_rur]

# Note: As there is no evidence for the bile metabolism to be active in HCT116 cell lines,
# the associated reactions were removed in the exometabo_constrain.ipynb file to reduce
# the overall model complexity. 

In [ ]:
# Load the thermodynamic database from Masid et al. 2020
thermo_data = load_thermoDB('../Data/thermo_data.thermodb')
# Load the metabolite concentrations with which to constrain
# the thermodynamic model 
met_conc = pickle.load(open("../Data/metabo_conc_abs.pkl","rb"))
# following names are not found in model
met_conc.pop('(R)-mevalonate') 
met_conc.pop('D-gluconic acid') 
met_conc.pop('ribose') 

In [ ]:
# Apply thermodynamic constraints to each of the model instances 
models = [model_wt,model_oxr,model_dmso,model_rur] #
names = ['Control','OxR','Control_DMSO','RuR'] #
tva_fluxes = []
for model,name in zip(models,names):
    model.solver = "gurobi"
    print(model.optimize())
    # Need to annotate the model to use the local thermodynamic database
    annotate_model_recon(model)
    annotate_model_kegg(model,disp=True) 
    copy_compartments_recon(model,name_length=0)
    solution = model.optimize()
    print(solution)
    # Convert model to thermodynamic model
    model.name = 'ThermoModel_{}'.format(name)
    apply_solver_settings(model,'optlang_gurobi')
    # Set up the thermodynamic model
    tmodel = create_thermo_model(thermo_data,model)
    # Apply the default thermodynamic constraints
    tmodel = met_conc_set_default(tmodel)
    # Constrain the thermodynamic model according to condition 
    apply_met_const(tmodel,met_conc,name)
    solution = tmodel.optimize()
    print(solution)
    add_pfba_constraint(tmodel)
    save_json_model(tmodel,"../Models/tmodel_{}.json".format(name))
    tva_fluxes.append(tfva(tmodel,name))